In [1]:
    import pandas as pd
    import numpy as np
    import json

# read logs

In [2]:
actions = pd.read_json("/Users/remim/tmp/all-2020.json")

In [3]:
actions.type.unique()

array(['unknown', 'themes', 'theme_candidates', 'home',
       'select_suggestion', 'suggestion_candidates', 'result_candidates',
       'search', 'select_result', 'visit_content', 'download', 'outlink',
       'external_search', 'modeles_courriers', 'outils', 'feedback'],
      dtype=object)

# read current suggestion file

In [356]:
with open('latest_entities_280120', 'r') as f :
    lines = f.readlines()

In [357]:
current_entities = pd.DataFrame([json.loads(e) for e in lines])

In [359]:
current_entities[:2]

,entity,value
0,congés familiaux,2
1,durée du préavis dans la boulangerie,2


# find frequent unsuggested request

In [4]:
search_actions = actions[actions.type == 'result_candidates']

In [403]:
all_queries = search_actions['query']
no_sugg_queries = all_queries[all_queries.apply(lambda x: str(x).lower() not in current_entities.entity.tolist())]


In [412]:
def riser(n):
    return np.around(15*np.log(n), 2)

In [440]:
selected = [
    'formation',
    'rupture conventionnelle',
    'astreinte',
    'formation',
    'cssct',
    'maladie',
    'repos compensateur',
    'incendie',
    'ancienneté',
    'allaitement',
    'préavis cdi',
    'formation cse',
    'clause de non concurrence',
    'repos hebdomadaire',
    'intéressement',
    'maternité',
    'repos',
    'repas',
    'stage',
    'invalidité',
    'syntec',
    'prime macron',
    'contrat',
    'handicap',
    'grossesse',
    "prise d'acte",
    'enfant malade',
    'mutuelle',
    'sécurité',
    'vestiaire',
    'document unique',
    'assistante maternelle',
    "taxe d'apprentissage",
    'prévoyance',
    'intérim',
    'risque chimique',
    'représentant syndical',
    'rgpd',
    'rupture',
    'naissance',
    'froid',
    'métallurgie',
    'mutation',
    'contrat de professionnalisation'
]

In [451]:
counts[:50]

undefined                                6773
rupture conventionnelle                   486
preavis                                   242
conges payes                              206
astreinte                                 166
formation                                 163
demission                                 161
conges                                    116
cssct                                     109
maladie                                   100
congé payé                                 90
congé                                      82
repos compensateur                         76
préavis cdi                                73
incendie                                   72
ancienneté                                 72
allaitement                                72
formation cse                              71
cong%c3%a9s%20pay%c3%a9s                   70
clause de non concurrence                  64
repos hebdomadaire                         64
congés payes                      

In [444]:
counts = no_sugg_queries.apply(lambda x : str(x).lower()).value_counts()
additions = [(s, 2+riser(counts.loc[s])) for s in selected]

In [448]:
for i in [json.dumps({ 'entity': e, 'value': v }) for (e,v) in additions]:
    print(i)

{"entity": "formation", "value": 78.41}
{"entity": "rupture conventionnelle", "value": 94.79}
{"entity": "astreinte", "value": 78.68}
{"entity": "formation", "value": 78.41}
{"entity": "cssct", "value": 72.37}
{"entity": "maladie", "value": 71.08}
{"entity": "repos compensateur", "value": 66.96}
{"entity": "incendie", "value": 66.15}
{"entity": "anciennet\u00e9", "value": 66.15}
{"entity": "allaitement", "value": 66.15}
{"entity": "pr\u00e9avis cdi", "value": 66.36}
{"entity": "formation cse", "value": 65.94}
{"entity": "clause de non concurrence", "value": 64.38}
{"entity": "repos hebdomadaire", "value": 64.38}
{"entity": "int\u00e9ressement", "value": 63.91}
{"entity": "maternit\u00e9", "value": 63.16}
{"entity": "repos", "value": 63.16}
{"entity": "repas", "value": 62.91}
{"entity": "stage", "value": 62.65}
{"entity": "invalidit\u00e9", "value": 61.27}
{"entity": "syntec", "value": 61.83}
{"entity": "prime macron", "value": 58.76}
{"entity": "contrat", "value": 58.76}
{"entity": "ha

In [408]:
current_entities[current_entities['entity'].apply(lambda x : 'prime' in x.lower())]['entity'].value_counts()

prime départ retraite                                                                                      1
une entreprise peut elle supprimer les tickets restaurant                                                  1
prime exceptionnelle                                                                                       1
qui a droit à la prime de précarité ?                                                                      1
calcul solde de tout compte et primes                                                                      1
prime                                                                                                      1
accident de travail et primes                                                                              1
prime de précarité convention banque cdd                                                                   1
versement immédiat de la prime de participation                                                            1
prime précarité    

In [426]:
'enfant malade' in current_entities.entity.tolist()

False

In [423]:
'disponibilité des droits des salariés' == 'disponibilité des droits des salariés'

True

In [ ]:
np.around(15*np.log(sugg_counts.values), 2)

# read logs and count suggestions

In [5]:
select_sugg_actions = actions[actions.type == 'select_suggestion']

In [6]:
select_sugg_actions['prefix_size'] = select_sugg_actions.prefix.apply(lambda p: len(p))

/Users/remim/dev/cdtn/.python-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
min_occ = 40

sugg_counts = select_sugg_actions['sugg_selection'].value_counts()

sugg_occ = sugg_counts.index[sugg_counts.values > min_occ].values

prefix_average = pd.Series(sugg_occ).apply(lambda s : round(select_sugg_actions[select_sugg_actions.sugg_selection == s].prefix_size.describe()['mean']))                                                

In [13]:
[print(f"{i} {sugg_counts[i]}") for i in sugg_occ[prefix_average[prefix_average > 10].index]]

rupture conventionnelle cdi 894
congés payés 512
heures supplémentaires 334
indemnités de licenciement 324
convention collective 319
licenciement économique 299
heures de délégation 223
entretien professionnel 170
affichage obligatoire 150
accident du travail 144
heures supplémentaires non payées 137
congé enfant malade 137
licenciement pour faute grave 127
congé maternité 120
règlement intérieur 111
délégué syndical 110
sécurité incendie 95
avenant au contrat de travail 94
rupture conventionnelle individuelle indemnité de rupture 91
jours de fractionnement 89
document unique d'évaluation des risques 89
durée du travail et temps de travail 89
augmentation de salaire 88
les congés pour événements familiaux 84
durée du préavis de démission 82
temps de trajet 78
indemnités frais de repas 77
prime de départ à la retraite 77
frais de déplacement trajet domicile travail 76
mutuelle santé obligatoire 75
contrat de travail 75
indemnité de départ à la retraite 73
travailleur de nuit 73
changeme

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [21]:
#exemple = "congés payés"
#exemple = "rupture conventionnelle cdi"
exemple = "heures supplémentaires"
print("count : " + str(sugg_counts[exemple]))
print("average prefix size : " + str(round(select_sugg_actions[select_sugg_actions.sugg_selection == exemple].prefix_size.describe()['mean'])))
select_sugg_actions[select_sugg_actions.sugg_selection == exemple]['prefix'][:30]

count : 334
average prefix size : 14.0


8278                heures suppléé
9836                     Heure sup
14327                    heures su
15659                   Heures sup
21231       heures supplémentaires
27423       heures supplémentaires
32421                  heures supp
33391                  heures supp
34580    rémunération heures suppl
45558                  Heures supp
47525                  heures supp
54643                   heures sup
71312       heures supplémentaires
71316                    heures su
72234       Heures supplémentaires
75169                    Heures su
77009                     heure su
86255       heures supplémentaires
89763                    heures su
89767                   heure supp
89788       heures supplémentaires
90961       heures supplémentaires
91247                heures supple
91901       heures supplementaires
91906       heures supplementaires
92350                    heures su
92470       heures supplémentaires
92477       heures supplémentaires
92644               

# update weights accordingly (with log to respect scale)

In [291]:
sugg_log = pd.DataFrame(index = sugg_counts.index.values, data = {'entity' : sugg_counts.index.values, 'log' : np.around(15*np.log(sugg_counts.values), 2), 'count' : sugg_counts.values})

In [292]:
sugg_log.loc['heures dif']

entity    heures dif
log             42.5
count             17
Name: heures dif, dtype: object

In [293]:
sugg_log.loc['heures supplémentaires']

entity    heures supplémentaires
log                        83.75
count                        266
Name: heures supplémentaires, dtype: object

In [294]:
augmentation = current_entities['entity'].apply(lambda x : sugg_log.loc[x]['log'] if x in sugg_log.index else 0)


In [295]:
current_entities['new_value'] = current_entities['value'] + augmentation
current_entities['count'] = current_entities['entity'].apply(lambda x : sugg_log.loc[x]['count'] if x in sugg_log.index else 0)



In [296]:
new_entities = current_entities[['entity', 'new_value']]

In [297]:
lines[:2]

['{"entity": "cong\\u00e9s familiaux", "value": 2}\n',
 '{"entity": "dur\\u00e9e du pr\\u00e9avis dans la boulangerie", "value": 2}\n']

In [318]:
new_entities['new_value'].describe()

count    2781.000000
mean       28.168382
std        19.513451
min         2.000000
25%        10.000000
50%        26.480000
75%        41.190000
max       110.920000
Name: new_value, dtype: float64

In [338]:
# add some non existing suggestion
new_entities = new_entities.append(pd.DataFrame([
    ['informer maternité', '10'],
    ['révéler maternité', '10']],
    columns=['entity', 'new_value']
))

In [340]:
# manual tweaks : patron drogue - ajouts - CDD

to_drop = new_entities[new_entities.entity == 'mon patron se drogue'].index[0]
new_entities.drop(to_drop, inplace=True)

entity = new_entities[new_entities.entity == 'cdd']
new_entities.set_value(entity.index[0], 'new_value', 50)
#new_entities[new_entities.entity == 'cdd']


IndexError: index 0 is out of bounds for axis 0 with size 0

In [331]:
# write back entities

In [341]:
new_lines = []
for _,row in new_entities.iterrows():
    obj = { 
        'entity': row['entity'],
        'value': row['new_value']
          }
    new_lines.append(json.dumps(obj))

In [342]:
raw = '\n'.join(new_lines)

In [343]:
with open('new_sugg', 'w+') as f:
    f.write(raw)